# Image Depth Project

In [5]:
import cv2
import numpy as np

## 1. Pixel wise matching

In [6]:
def distance(x, y, distance_type):
    if distance_type == "l1":
        return abs(x - y)
    elif distance_type == "l2":
        return (x - y)**2

def pixel_wise_matching(left_img, right_img, disparity_range, distance_type, save_result = True):
    # Read left, right images then convert to grayscale
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape

    # Create blank disparity map
    depth = np.zeros((height, width), np.uint8)
    scale = 16
    max_value = 255

    for y in range(height):
        for x in range(width):
            # Find j where cost has minimum value in horizontal
            disparity = 0
            cost_min = max_value

            for j in range(disparity_range):
                cost = max_value if (x - j) < 0 else distance(int(left[y, x]), int(right[y, x - j]), distance_type)

                if cost < cost_min :
                    cost_min = cost
                    disparity = j

            # Multiply by a scale factor for visualization purpose
            depth[y, x] = disparity * scale

    if save_result == True :
        print("Saving result...")
        # Save results
        cv2.imwrite (f"pixel_wise_{distance_type}.png", depth)
        cv2.imwrite (f"pixel_wise_{distance_type}_color.png", cv2.applyColorMap(depth, cv2.COLORMAP_JET))

    print("Done.")

    return depth

In [7]:
left_img_path = "tsukuba/left.png"
right_img_path = "tsukuba/right.png"
disparity_range = 16

# L1
pixel_wise_result_l1 = pixel_wise_matching(left_img_path, right_img_path, disparity_range, "l1", save_result = True)

# L2
pixel_wise_result_l2 = pixel_wise_matching(left_img_path, right_img_path, disparity_range, "l2", save_result = True)

Saving result...
Done.
Saving result...
Done.


## 2. Window based matching

In [9]:
def window_based_matching(left_img, right_img, disparity_range, distance_type, kernel_size = 5, save_result = True):
    # Read left , right images then convert to grayscale
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape

    # Create blank disparity map
    depth = np.zeros((height, width ), np.uint8)

    kernel_half = int((kernel_size - 1) / 2)
    scale = 3
    max_value = 255 * 9

    for y in range(kernel_half, height - kernel_half):
        for x in range(kernel_half, width - kernel_half):
            # Find j where cost has minimum value
            disparity = 0
            cost_min = 65534

            for j in range(disparity_range):
                total = 0
                value = 0

                for v in range(-kernel_half, kernel_half + 1):
                    for u in range (-kernel_half, kernel_half + 1):
                        value = max_value
                        if (x + u - j) >= 0:
                            value = distance(int(left[y + v, x + u]), int(right[y + v, (x + u) - j]), distance_type)
                            total += value

                if total < cost_min:
                    cost_min = total
                    disparity = j

            # Multiply by a scale factor for visualization purpose
            depth[y, x] = disparity * scale

    if save_result == True :
        print("Saving result...")
        # Save results
        cv2.imwrite(f"window_based_{distance_type}.png", depth)
        cv2.imwrite(f"window_based_{distance_type}_color.png", cv2.applyColorMap(depth, cv2.COLORMAP_JET))

    print("Done.")

    return depth

In [10]:
left_img_path = "Aloe/Aloe_left_1.png"
right_img_path = "Aloe/Aloe_right_1.png"
disparity_range = 64
kernel_size = 3

# L1
pixel_wise_result_l1 = window_based_matching(left_img_path, right_img_path, disparity_range, "l1", kernel_size = kernel_size, save_result = True)

# L2
pixel_wise_result_l2 = window_based_matching(left_img_path, right_img_path, disparity_range, "l2", kernel_size = kernel_size , save_result = True)

Saving result...
Done.
Saving result...
Done.


## 3. Window based matching with different brightness

In [13]:
def cosine_similarity(x, y):
    numerator = np.dot(x, y)
    denominator = np.linalg.norm(x) * np.linalg.norm(y)

    return numerator / denominator

def window_based_matching_different_brightness(left_img, right_img, disparity_range, kernel_size = 5, img_type = "high", save_result = True):
    # Read left , right images then convert to grayscale
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape

    # Create blank disparity map
    depth = np.zeros((height, width), np.uint8)
    kernel_half = int((kernel_size - 1) / 2)
    scale = 3

    for y in range(kernel_half, height - kernel_half):
        for x in range(kernel_half, width - kernel_half):
            # Find j where cost has minimum value
            disparity = 0
            cost_optimal = -1

            for j in range(disparity_range):
                d = x - j
                cost = -1
                if (d - kernel_half) > 0:
                    wp = left[(y - kernel_half):(y + kernel_half) + 1, (x - kernel_half):(x + kernel_half) + 1]
                    wqd = right[(y - kernel_half):(y + kernel_half) + 1, (d - kernel_half):(d + kernel_half) + 1]

                    wp_flattened = wp.flatten()
                    wqd_flattened = wqd .flatten()

                    cost = cosine_similarity(wp_flattened, wqd_flattened)

                if cost > cost_optimal:
                    cost_optimal = cost
                    disparity = j

            # Multiply by a scale factor for visualization purpose
            depth[y, x] = disparity * scale

    if save_result == True :
        print("Saving result...")
        # Save results
        cv2.imwrite(f"window_based_cosine_similarity_{img_type}_brightness.png", depth)
        cv2.imwrite(f"window_based_cosine_similarity_color_{img_type}_brightness.png", cv2.applyColorMap(depth, cv2.COLORMAP_JET))

    print("Done.")

    return depth

In [14]:
# High brightness
left_img_path = "Aloe/Aloe_left_1.png"
right_img_path = "Aloe/Aloe_right_2.png"
disparity_range = 64
kernel_size = 3

# Cosine similarity
pixel_wise_result_cosine = window_based_matching_different_brightness(left_img_path, right_img_path, disparity_range, kernel_size = kernel_size, img_type = "high", save_result = True)

Saving result...
Done.


In [15]:
# Low brightness
left_img_path = "Aloe/Aloe_left_1.png"
right_img_path = "Aloe/Aloe_right_3.png"
disparity_range = 64
kernel_size = 3

# Cosine similarity
pixel_wise_result_cosine = window_based_matching_different_brightness(left_img_path, right_img_path, disparity_range, kernel_size = kernel_size, img_type = "low", save_result = True)

Saving result...
Done.
